# Notebook 1 -- Nettoyage des donnees et analyse exploratoire

## Contexte

Ce notebook constitue la premiere etape de notre analyse des scores ESG (Environmental, Social, Governance) 
pour un panel de 23 entreprises technologiques et innovantes cotees en bourse. Les donnees proviennent 
du terminal Bloomberg (scores BESG) et couvrent la periode 2016-2026 avec une frequence mensuelle.

L'objectif est de reproduire et d'ameliorer la methodologie presentee dans le paper de reference 
("K-means et analyse de clustering hierarchique agglomeratif des scores ESG") en l'appliquant 
a un univers d'investissement different : les entreprises liees a l'intelligence artificielle et 
aux semi-conducteurs, plutot que le secteur de l'energie.

## Structure de ce notebook

1. Chargement et inspection des donnees brutes
2. Nettoyage : conversion des formats (dates Excel, separateurs decimaux, valeurs manquantes)
3. Restructuration en format exploitable (panel long)
4. Statistiques descriptives
5. Analyse des valeurs manquantes
6. Visualisations exploratoires (evolution temporelle, distributions, correlations)

## Fichiers sources

- `data/ENVIRONMENTAL_SCORE.csv` : BESG Environmental Pillar Score
- `data/SOCIAL_SCORE.csv` : BESG Social Pillar Score
- `data/esg score.csv` : BESG ESG Score (score composite)

---
## 1. Imports et configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Configuration graphique
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
sns.set_style('whitegrid')

# Chemin vers les donnees
DATA_DIR = '../data/'

print('Imports effectues avec succes.')

---
## 2. Chargement des donnees brutes

Les fichiers CSV exportes depuis Bloomberg ont un format particulier :
- Les 10 premieres lignes contiennent des metadonnees (titre, dates de debut/fin, tickers, noms de champs)
- Les donnees effectives commencent a la ligne 11
- La premiere colonne est vide, la deuxieme contient les dates au format numerique Excel
- Les scores utilisent la virgule comme separateur decimal (format francais)
- Les valeurs manquantes sont codees `#N/A N/A`

In [ ]:
def charger_csv_bloomberg(filepath, nom_score):
    """
    Charge un fichier CSV exporte depuis Bloomberg et le nettoie.
    
    Parametres
    ----------
    filepath : str
        Chemin vers le fichier CSV.
    nom_score : str
        Nom du type de score (ex: 'Environmental', 'Social', 'ESG').
    
    Retourne
    --------
    pd.DataFrame
        DataFrame nettoye en format long avec colonnes : Date, Ticker, Score, Type.
    """
    # Lecture brute : on saute les 9 premieres lignes de metadonnees
    # La ligne 10 (index 9) contient 'Dates' et les noms de colonnes
    raw = pd.read_csv(
        filepath,
        header=None,
        skiprows=10,
        na_values=['#N/A N/A', '#N/A', 'N/A', ''],
        encoding='utf-8'
    )
    
    # Lecture de la ligne d'en-tete avec les tickers (ligne 8, index 7)
    header_raw = pd.read_csv(filepath, header=None, nrows=10, encoding='utf-8')
    tickers = header_raw.iloc[7, 2:].values  # Les tickers commencent a la colonne 2
    
    # Construction du DataFrame
    # Colonne 0 : vide, Colonne 1 : Dates, Colonnes 2+ : scores par entreprise
    df = raw.iloc[:, 1:].copy()
    df.columns = ['Date_Excel'] + list(tickers)
    
    print(f"[{nom_score}] Dimensions brutes : {df.shape[0]} lignes x {df.shape[1]-1} entreprises")
    print(f"[{nom_score}] Periode : date Excel {df['Date_Excel'].iloc[0]} a {df['Date_Excel'].iloc[-1]}")
    
    return df, tickers


# Chargement des trois fichiers
df_env_raw, tickers_env = charger_csv_bloomberg(DATA_DIR + 'ENVIRONMENTAL_SCORE.csv', 'Environmental')
df_soc_raw, tickers_soc = charger_csv_bloomberg(DATA_DIR + 'SOCIAL_SCORE.csv', 'Social')
df_esg_raw, tickers_esg = charger_csv_bloomberg(DATA_DIR + 'esg score.csv', 'ESG')

In [ ]:
# Verification : les tickers sont-ils identiques dans les 3 fichiers ?
print("Tickers Environmental :", list(tickers_env))
print("\nTickers Social :", list(tickers_soc))
print("\nTickers ESG :", list(tickers_esg))
print("\nTickers identiques dans les 3 fichiers :", 
      np.array_equal(tickers_env, tickers_soc) and np.array_equal(tickers_soc, tickers_esg))

In [ ]:
# Apercu des donnees brutes (avant nettoyage)
print("=== Apercu Environmental Score (brut) ===")
df_env_raw.head()

---
## 3. Nettoyage des donnees

### 3.1 Conversion des dates

Les dates sont au format numerique Excel (nombre de jours depuis le 30 decembre 1899). 
Par exemple, 42369 correspond au 1er janvier 2016.

### 3.2 Conversion des separateurs decimaux

Les scores utilisent la virgule comme separateur decimal (ex: `"3,43"` pour 3.43). 
Il faut remplacer les virgules par des points puis convertir en float.

### 3.3 Traitement des valeurs manquantes

Les `#N/A N/A` de Bloomberg indiquent que le score n'etait pas encore disponible pour cette entreprise 
a cette date. C'est un phenomene courant : les scores ESG ne sont pas calcules des l'introduction 
en bourse, ils apparaissent progressivement au fil du temps.

In [ ]:
def excel_date_to_datetime(excel_date):
    """
    Convertit une date au format numerique Excel en datetime Python.
    La base Excel est le 30 decembre 1899 (convention Windows).
    """
    base = datetime(1899, 12, 30)
    return base + timedelta(days=int(excel_date))


def nettoyer_scores(df, nom_score):
    """
    Nettoie un DataFrame brut de scores Bloomberg :
    - Convertit les dates Excel en datetime
    - Remplace les virgules par des points dans les scores
    - Convertit les scores en float
    - Gere les valeurs manquantes
    
    Parametres
    ----------
    df : pd.DataFrame
        DataFrame brut avec Date_Excel + colonnes de scores.
    nom_score : str
        Type de score pour le nommage.
    
    Retourne
    --------
    pd.DataFrame
        DataFrame nettoye avec dates converties et scores en float.
    """
    df_clean = df.copy()
    
    # Conversion des dates
    df_clean['Date'] = df_clean['Date_Excel'].apply(excel_date_to_datetime)
    df_clean = df_clean.drop(columns=['Date_Excel'])
    
    # Conversion des scores : remplacer virgule par point puis convertir en float
    score_cols = [c for c in df_clean.columns if c != 'Date']
    for col in score_cols:
        # Certaines valeurs sont deja numeriques (int ou float), d'autres sont des strings
        df_clean[col] = df_clean[col].apply(
            lambda x: float(str(x).replace(',', '.')) if pd.notna(x) and str(x).strip() != '' else np.nan
        )
    
    # Reordonner les colonnes
    df_clean = df_clean[['Date'] + score_cols]
    
    # Statistiques de nettoyage
    n_total = df_clean[score_cols].size
    n_missing = df_clean[score_cols].isna().sum().sum()
    pct_missing = 100 * n_missing / n_total
    print(f"[{nom_score}] Apres nettoyage : {n_missing}/{n_total} valeurs manquantes ({pct_missing:.1f}%)")
    print(f"[{nom_score}] Periode : {df_clean['Date'].min().strftime('%Y-%m-%d')} a {df_clean['Date'].max().strftime('%Y-%m-%d')}")
    
    return df_clean


# Application du nettoyage
df_env = nettoyer_scores(df_env_raw, 'Environmental')
df_soc = nettoyer_scores(df_soc_raw, 'Social')
df_esg = nettoyer_scores(df_esg_raw, 'ESG')

In [ ]:
# Apercu apres nettoyage
print("=== Environmental Score (nettoye) ===")
df_env.head()

In [ ]:
print("=== Social Score (nettoye) ===")
df_soc.head()

In [ ]:
print("=== ESG Score global (nettoye) ===")
df_esg.head()

---
## 4. Restructuration en format long (panel)

Pour faciliter les analyses statistiques et les visualisations, on transforme les DataFrames 
du format large (une colonne par entreprise) au format long (une ligne par observation date/entreprise). 
On fusionne ensuite les trois types de scores dans un seul DataFrame panel.

In [ ]:
def wide_to_long(df, score_type):
    """
    Transforme un DataFrame wide en format long.
    
    Parametres
    ----------
    df : pd.DataFrame
        DataFrame en format large (Date + colonnes de scores par ticker).
    score_type : str
        Nom du type de score.
    
    Retourne
    --------
    pd.DataFrame
        Format long avec colonnes : Date, Ticker, Score.
    """
    score_cols = [c for c in df.columns if c != 'Date']
    df_long = df.melt(id_vars='Date', value_vars=score_cols,
                      var_name='Ticker', value_name=score_type)
    return df_long


# Transformation en format long
df_env_long = wide_to_long(df_env, 'Env_Score')
df_soc_long = wide_to_long(df_soc, 'Soc_Score')
df_esg_long = wide_to_long(df_esg, 'ESG_Score')

# Fusion des trois scores dans un seul DataFrame panel
df_panel = df_env_long.merge(df_soc_long, on=['Date', 'Ticker'], how='outer')
df_panel = df_panel.merge(df_esg_long, on=['Date', 'Ticker'], how='outer')

# Tri par ticker puis par date
df_panel = df_panel.sort_values(['Ticker', 'Date']).reset_index(drop=True)

print(f"Dimensions du panel : {df_panel.shape}")
print(f"Nombre d'entreprises : {df_panel['Ticker'].nunique()}")
print(f"Nombre de dates : {df_panel['Date'].nunique()}")
df_panel.head(10)

In [ ]:
# Dictionnaire pour identifier les entreprises
TICKER_NAMES = {
    'NVDA US Equity': 'NVIDIA',
    'AVGO US Equity': 'Broadcom',
    'TSM US Equity': 'TSMC',
    'GOOGL US Equity': 'Alphabet (Google)',
    '000660 KS Equity': 'SK Hynix',
    'LRCX US Equity': 'Lam Research',
    '6857 JP Equity': 'Advantest',
    'TSEM IT Equity': 'Tower Semiconductor',
    'MSFT US Equity': 'Microsoft',
    'LITE US Equity': 'Lumentum',
    'AMD US Equity': 'AMD',
    'FN US Equity': 'Fabrinet',
    'SNOW US Equity': 'Snowflake',
    'MU US Equity': 'Micron Technology',
    'TSLA US Equity': 'Tesla',
    '9984 JP Equity': 'SoftBank',
    'CRDO US Equity': 'CREDO Technology',
    'ENR GR Equity': 'Siemens Energy',
    'MPWR US Equity': 'Monolithic Power Systems',
    '2383 TT Equity': 'Elite Material',
    'CLS CN Equity': 'Celestica',
    'META US Equity': 'Meta Platforms',
    'AMZN US Equity': 'Amazon'
}

df_panel['Entreprise'] = df_panel['Ticker'].map(TICKER_NAMES)
print("Entreprises du panel :")
for ticker, name in sorted(TICKER_NAMES.items(), key=lambda x: x[1]):
    print(f"  {ticker:25s} -> {name}")

---
## 5. Analyse des valeurs manquantes

L'analyse des valeurs manquantes est fondamentale avant toute modelisation. 
Dans le contexte des scores ESG Bloomberg, les valeurs manquantes ne sont generalement pas 
aleatoires (MCAR) mais structurelles : elles correspondent a des periodes ou l'entreprise 
n'avait pas encore de score BESG attribue.

On distingue :
- Les **manquantes en debut de serie** (left-censoring) : le score n'existait pas encore
- Les **manquantes sporadiques** : potentiellement des erreurs ou des periodes de recalcul

In [ ]:
# Matrice de valeurs manquantes pour chaque type de score (format wide)
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for ax, (df_w, titre) in zip(axes, [
    (df_env, 'Environmental Score'),
    (df_soc, 'Social Score'),
    (df_esg, 'ESG Score Global')
]):
    score_cols = [c for c in df_w.columns if c != 'Date']
    # Matrice binaire : 1 = present, 0 = manquant
    presence = df_w[score_cols].notna().astype(int)
    # Renommer les colonnes pour lisibilite
    presence.columns = [TICKER_NAMES.get(c, c).split('(')[0].strip()[:12] for c in presence.columns]
    
    sns.heatmap(presence.T, cmap='YlGn', cbar_kws={'label': 'Donnee presente'},
                xticklabels=20, ax=ax)
    ax.set_xlabel('Indice temporel')
    ax.set_ylabel('')
    ax.set_title(f'Disponibilite -- {titre}')

plt.tight_layout()
plt.savefig('../notebooks/figures/01_missing_data_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print("Figure sauvegardee : figures/01_missing_data_heatmap.png")

In [ ]:
# Pourcentage de valeurs manquantes par entreprise
missing_summary = pd.DataFrame()
for df_w, nom in [(df_env, 'Env'), (df_soc, 'Soc'), (df_esg, 'ESG')]:
    score_cols = [c for c in df_w.columns if c != 'Date']
    pct = df_w[score_cols].isna().mean() * 100
    missing_summary[nom] = pct

missing_summary.index = [TICKER_NAMES.get(c, c) for c in missing_summary.index]
missing_summary['Moyenne'] = missing_summary.mean(axis=1)
missing_summary = missing_summary.sort_values('Moyenne', ascending=False)

print("Pourcentage de valeurs manquantes par entreprise (%) :")
print(missing_summary.round(1).to_string())

In [ ]:
# Date de premiere apparition du score pour chaque entreprise
print("\nDate de premiere observation disponible par entreprise et par score :\n")
first_dates = pd.DataFrame()
for df_w, nom in [(df_env, 'Env'), (df_soc, 'Soc'), (df_esg, 'ESG')]:
    score_cols = [c for c in df_w.columns if c != 'Date']
    first = {}
    for col in score_cols:
        valid = df_w.loc[df_w[col].notna(), 'Date']
        first[col] = valid.iloc[0].strftime('%Y-%m') if len(valid) > 0 else 'N/A'
    first_dates[nom] = pd.Series(first)

first_dates.index = [TICKER_NAMES.get(c, c) for c in first_dates.index]
print(first_dates.to_string())

---
## 6. Statistiques descriptives

On calcule les statistiques descriptives classiques (moyenne, ecart-type, min, max, quartiles) 
pour chaque type de score. Cette etape permet de detecter d'eventuelles anomalies 
(scores hors echelle, distributions atypiques) et de mieux comprendre la structure des donnees.

In [ ]:
# Statistiques descriptives globales
print("=" * 60)
print("STATISTIQUES DESCRIPTIVES DU PANEL")
print("=" * 60)

for score_col, titre in [('Env_Score', 'Environmental'), ('Soc_Score', 'Social'), ('ESG_Score', 'ESG Global')]:
    print(f"\n--- {titre} ---")
    print(df_panel[score_col].describe().round(3).to_string())

In [ ]:
# Statistiques descriptives par entreprise (derniere observation disponible)
# On prend le dernier score disponible pour chaque entreprise comme proxy de l'etat actuel
latest_scores = df_panel.groupby('Ticker').last().reset_index()
latest_scores['Entreprise'] = latest_scores['Ticker'].map(TICKER_NAMES)

summary_table = latest_scores[['Entreprise', 'Env_Score', 'Soc_Score', 'ESG_Score']].copy()
summary_table = summary_table.sort_values('ESG_Score', ascending=False).reset_index(drop=True)
summary_table.index = summary_table.index + 1  # Index a partir de 1

print("\nClassement des entreprises par score ESG global (derniere observation) :")
print(summary_table.to_string())

In [ ]:
# Distribution des scores (histogrammes + KDE)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (col, titre, couleur) in zip(axes, [
    ('Env_Score', 'Environmental Score', '#2ca02c'),
    ('Soc_Score', 'Social Score', '#1f77b4'),
    ('ESG_Score', 'ESG Score Global', '#d62728')
]):
    data = df_panel[col].dropna()
    ax.hist(data, bins=30, density=True, alpha=0.6, color=couleur, edgecolor='white')
    data.plot.kde(ax=ax, color=couleur, linewidth=2)
    ax.axvline(data.mean(), color='black', linestyle='--', linewidth=1.5, label=f'Moyenne = {data.mean():.2f}')
    ax.axvline(data.median(), color='gray', linestyle=':', linewidth=1.5, label=f'Mediane = {data.median():.2f}')
    ax.set_xlabel('Score')
    ax.set_ylabel('Densite')
    ax.set_title(f'Distribution -- {titre}')
    ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('../notebooks/figures/01_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. Visualisation de l'evolution temporelle

L'evolution temporelle des scores ESG permet de detecter des tendances structurelles 
(amelioration ou degradation) ainsi que des ruptures potentielles 
(changement de methodologie Bloomberg, evenement externe).

In [ ]:
# Evolution temporelle de toutes les entreprises pour chaque score
fig, axes = plt.subplots(3, 1, figsize=(16, 18))

for ax, (df_w, titre, cmap_name) in zip(axes, [
    (df_env, 'Environmental Score', 'Greens'),
    (df_soc, 'Social Score', 'Blues'),
    (df_esg, 'ESG Score Global', 'Reds')
]):
    score_cols = [c for c in df_w.columns if c != 'Date']
    cmap = plt.cm.get_cmap(cmap_name, len(score_cols))
    
    for i, col in enumerate(score_cols):
        short_name = TICKER_NAMES.get(col, col).split('(')[0].strip()
        ax.plot(df_w['Date'], df_w[col], label=short_name, 
                color=cmap(i / len(score_cols)), alpha=0.8, linewidth=1.2)
    
    ax.set_ylabel('Score BESG')
    ax.set_title(f'Evolution temporelle -- {titre}')
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=7, ncol=2)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_major_locator(mdates.YearLocator())

plt.tight_layout()
plt.savefig('../notebooks/figures/01_evolution_temporelle.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Evolution de la moyenne et de l'ecart-type du panel au cours du temps
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

for df_w, titre, color in [
    (df_env, 'Environmental', '#2ca02c'),
    (df_soc, 'Social', '#1f77b4'),
    (df_esg, 'ESG Global', '#d62728')
]:
    score_cols = [c for c in df_w.columns if c != 'Date']
    mean_ts = df_w[score_cols].mean(axis=1)
    std_ts = df_w[score_cols].std(axis=1)
    
    axes[0].plot(df_w['Date'], mean_ts, label=titre, color=color, linewidth=2)
    axes[0].fill_between(df_w['Date'], mean_ts - std_ts, mean_ts + std_ts, 
                          alpha=0.15, color=color)
    axes[1].plot(df_w['Date'], std_ts, label=titre, color=color, linewidth=2)

axes[0].set_ylabel('Score moyen du panel')
axes[0].set_title('Moyenne des scores ESG du panel (bande = +/- 1 ecart-type)')
axes[0].legend()
axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

axes[1].set_ylabel('Ecart-type')
axes[1].set_title('Dispersion (ecart-type) des scores au sein du panel')
axes[1].legend()
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.tight_layout()
plt.savefig('../notebooks/figures/01_mean_std_evolution.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 8. Analyse des correlations

L'etude des correlations entre les trois piliers ESG permet de verifier si les scores 
sont redondants (forte correlation) ou apportent chacun une information distincte. 
Cela a des implications directes pour la strategie de clustering : si les piliers sont 
tres correles, une reduction de dimension (ACP) sera pertinente.

In [ ]:
# Correlation entre les trois types de scores
corr_scores = df_panel[['Env_Score', 'Soc_Score', 'ESG_Score']].corr()

fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(corr_scores, annot=True, fmt='.3f', cmap='RdYlBu_r', center=0,
            vmin=-1, vmax=1, square=True, ax=ax,
            xticklabels=['Environmental', 'Social', 'ESG Global'],
            yticklabels=['Environmental', 'Social', 'ESG Global'])
ax.set_title('Matrice de correlation entre les piliers ESG')

plt.tight_layout()
plt.savefig('../notebooks/figures/01_correlation_piliers.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nMatrice de correlation :")
print(corr_scores.round(4).to_string())

In [ ]:
# Scatter plots entre les piliers
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

pairs = [
    ('Env_Score', 'Soc_Score', 'Environnemental vs Social'),
    ('Env_Score', 'ESG_Score', 'Environnemental vs ESG Global'),
    ('Soc_Score', 'ESG_Score', 'Social vs ESG Global')
]

for ax, (x_col, y_col, title) in zip(axes, pairs):
    subset = df_panel.dropna(subset=[x_col, y_col])
    ax.scatter(subset[x_col], subset[y_col], alpha=0.3, s=10, color='steelblue')
    
    # Droite de regression
    z = np.polyfit(subset[x_col], subset[y_col], 1)
    p = np.poly1d(z)
    x_range = np.linspace(subset[x_col].min(), subset[x_col].max(), 100)
    ax.plot(x_range, p(x_range), 'r--', linewidth=2, 
            label=f'y = {z[0]:.2f}x + {z[1]:.2f}')
    
    ax.set_xlabel(x_col.replace('_', ' '))
    ax.set_ylabel(y_col.replace('_', ' '))
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.savefig('../notebooks/figures/01_scatter_piliers.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation inter-entreprises (sur le score ESG global)
# Utilise le format wide : une colonne par entreprise
score_cols = [c for c in df_esg.columns if c != 'Date']
corr_entreprises = df_esg[score_cols].corr()

# Renommer pour lisibilite
short_names = [TICKER_NAMES.get(c, c).split('(')[0].strip()[:10] for c in corr_entreprises.columns]
corr_entreprises.columns = short_names
corr_entreprises.index = short_names

fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_entreprises, dtype=bool), k=1)
sns.heatmap(corr_entreprises, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, vmin=-1, vmax=1, ax=ax, annot_kws={'size': 7})
ax.set_title('Matrice de correlation inter-entreprises (Score ESG Global)')

plt.tight_layout()
plt.savefig('../notebooks/figures/01_correlation_entreprises.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 9. Calcul des variations annuelles

Comme dans le paper de reference, on calcule les variations annuelles des scores ESG. 
Ces variations capturent la dynamique d'amelioration ou de degradation des pratiques ESG, 
ce qui est une information complementaire au niveau absolu du score.

On calcule la variation en difference absolue (delta) plutot qu'en pourcentage, 
car les scores BESG sont deja sur une echelle bornee [0, 10].

In [ ]:
def calculer_variations_annuelles(df_wide):
    """
    Calcule les variations annuelles (delta) pour chaque entreprise.
    On prend la difference entre le score de la meme periode d'une annee a l'autre (lag 12 mois).
    
    Parametres
    ----------
    df_wide : pd.DataFrame
        DataFrame en format wide avec colonne 'Date' et colonnes de scores.
    
    Retourne
    --------
    pd.DataFrame
        DataFrame des variations annuelles, meme format.
    """
    score_cols = [c for c in df_wide.columns if c != 'Date']
    df_var = df_wide.copy()
    
    for col in score_cols:
        df_var[col] = df_wide[col].diff(periods=12)  # Diff sur 12 mois
    
    return df_var


df_env_var = calculer_variations_annuelles(df_env)
df_soc_var = calculer_variations_annuelles(df_soc)
df_esg_var = calculer_variations_annuelles(df_esg)

# Statistiques des variations
print("Statistiques des variations annuelles du score ESG Global :")
score_cols = [c for c in df_esg_var.columns if c != 'Date']
var_stats = df_esg_var[score_cols].describe()
var_stats.columns = [TICKER_NAMES.get(c, c).split('(')[0].strip()[:12] for c in var_stats.columns]
print(var_stats.round(3).to_string())

In [ ]:
# Boxplot des variations annuelles par entreprise
fig, ax = plt.subplots(figsize=(16, 6))

score_cols = [c for c in df_esg_var.columns if c != 'Date']
var_data = df_esg_var[score_cols].dropna()
var_data.columns = [TICKER_NAMES.get(c, c).split('(')[0].strip()[:10] for c in var_data.columns]

var_data.boxplot(ax=ax, vert=True, patch_artist=True,
                 boxprops=dict(facecolor='lightsteelblue', color='navy'),
                 medianprops=dict(color='red', linewidth=2))
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_ylabel('Variation annuelle du score ESG')
ax.set_title('Distribution des variations annuelles du score ESG par entreprise')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig('../notebooks/figures/01_variations_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 10. Sauvegarde des donnees nettoyees

On sauvegarde les donnees nettoyees en format CSV pour les reutiliser dans les notebooks suivants. 
Cela evite de repeter le processus de nettoyage et garantit la coherence entre les etapes.

In [ ]:
import os

# Creation du repertoire de sortie
os.makedirs('../data/clean', exist_ok=True)
os.makedirs('../notebooks/figures', exist_ok=True)

# Sauvegarde en format wide (une colonne par entreprise)
df_env.to_csv('../data/clean/env_score_clean.csv', index=False)
df_soc.to_csv('../data/clean/soc_score_clean.csv', index=False)
df_esg.to_csv('../data/clean/esg_score_clean.csv', index=False)

# Sauvegarde des variations annuelles
df_env_var.to_csv('../data/clean/env_score_variations.csv', index=False)
df_soc_var.to_csv('../data/clean/soc_score_variations.csv', index=False)
df_esg_var.to_csv('../data/clean/esg_score_variations.csv', index=False)

# Sauvegarde du panel complet
df_panel.to_csv('../data/clean/panel_complet.csv', index=False)

print("Fichiers sauvegardes dans data/clean/ :")
for f in sorted(os.listdir('../data/clean')):
    size = os.path.getsize(f'../data/clean/{f}')
    print(f"  {f:40s} ({size/1024:.1f} Ko)")

---
## Synthese du Notebook 1

### Donnees
- **Panel** : 23 entreprises technologiques/innovantes, 122 observations mensuelles (2016-2026)
- **Scores** : BESG Environmental Pillar, BESG Social Pillar, BESG ESG Score (composite)
- **Source** : Bloomberg Terminal

### Observations principales
- Les valeurs manquantes sont concentrees en debut de serie (left-censoring) : plusieurs entreprises n'avaient pas de score ESG avant 2017-2020
- Les scores sont sur une echelle 0-10, avec une tendance generale a la hausse sur la periode
- La dispersion entre entreprises est significative, ce qui justifie l'application de methodes de clustering
- Les correlations entre piliers Environmental et Social sont a examiner pour justifier ou non l'utilisation des deux dimensions separement

### Prochaines etapes
- **Notebook 2** : Application du K-means et du clustering hierarchique agglomeratif (CAH)
- **Notebook 3** : Ameliorations methodologiques (ACP, DBSCAN, GMM, analyse temporelle)